In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import sqlite3
from werkzeug.security import generate_password_hash
from datetime import datetime

app = Flask(__name__)

# Initialize the SQLite database
def init_db():
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    # Create users table if it doesn't exist
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  first_name TEXT NOT NULL,
                  last_name TEXT NOT NULL,
                  email TEXT NOT NULL,
                  password TEXT NOT NULL,
                  created_at TEXT NOT NULL)''')
    conn.commit()
    conn.close()

@app.route('/')
def home():
    return '''
    <h1> Patient Database </h1>
    <a href="register">Go to registration</a><br>
    <a href="users">View users</a><br>
    '''

# Route to display the registration form
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        # Retrieve form data
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        email = request.form['email']
        password = request.form['password']

        # Hash the password
        # hashed_password = generate_password_hash(password, method='sha256')
        hashed_password = generate_password_hash(password, method='pbkdf2:sha256')

        # Get the current timestamp
        created_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Insert user data into the database
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        c.execute('''INSERT INTO users (first_name, last_name, email, password, created_at) 
                     VALUES (?, ?, ?, ?, ?)''', 
                  (first_name, last_name, email, hashed_password, created_at))
        conn.commit()
        conn.close()

        # Redirect to the success page
        return redirect(url_for('success', first_name=first_name))

    return render_template('register.html')

# Route to display the success page
@app.route('/success/<first_name>')
def success(first_name):
    return f"User {first_name} registered successfully!\n<a href=\"/\">Go to homepage</a>"

# Route to list all users (excluding passwords)
@app.route('/users')
def list_users():
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('SELECT id, first_name, last_name, email, created_at FROM users')
    users = c.fetchall()
    conn.close()
    return render_template('users.html', users=users)

# Route to delete a user by ID
@app.route('/delete/<int:user_id>')
def delete_user(user_id):
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('DELETE FROM users WHERE id = ?', (user_id,))
    conn.commit()
    conn.close()
    return redirect(url_for('list_users'))

# Initialize the database and run the Flask app
if __name__ == '__main__':
    init_db()
    app.run(port=5500, debug=False)  # Disable debug mode